<a href="https://colab.research.google.com/github/leo-aguiar/sistema-de-validacao-de-transacoes/blob/main/Projeto_Valida%C3%A7%C3%A3o_de_Transa%C3%A7%C3%B5es_Empresariais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
O projeto desenvolvido em Python visa validar transações de uma empresa a partir de arquivos CSV. Esses arquivos contêm informações sobre 100 transações
recentes, incluindo CPF, número do cliente, valor da transação e produto transacionado. Além disso, são fornecidos arquivos com listas de CPFs e números
de clientes válidos para comparação.

O objetivo principal é criar um relatório que indique a quantidade de transações válidas e inválidas, bem como detalhar os erros encontrados em cada
transação inválida. Para isso, foi desenvolvida a função valida_transacao(), que verifica se o CPF e o número do cliente estão nas listas válidas e se o
valor da transação é positivo. Essa função retorna verdadeiro somente se todas as condições forem satisfeitas, garantindo a precisão na validação dos
dados.

Durante a validação, são considerados seis tipos de erros: CPF não numérico, número do cliente não numérico, valor da transação não numérico, produto
não string, produto diferente de "Streaming" ou "Delivery", e transação inválida (quando a função retorna falso). As transações válidas e inválidas são
armazenadas em listas separadas para facilitar a organização e análise posterior.

Ao final do processo, o relatório gerado em formato txt detalha a quantidade de transações válidas e inválidas, incluindo mensagens específicas para cada
erro encontrado.

Durante o desenvolvimento deste projeto, foram exercitadas habilidades como manipulação de arquivos, uso de listas e funções em Python, além da aplicação
de técnicas de processamento e validação de dados, assegurando a integridade e qualidade dos dados no sistema.
'''

In [1]:
# Importa as bibliotecas [pandas e numpy]
import pandas as pd
import numpy as np

In [2]:
# Lê a coluna 'cpf' do arquivo CSV e converte para uma lista
cpfs = pd.read_csv('cpfs_elegiveis.csv')['cpf'].tolist()

In [3]:
# Lê a coluna 'nu_cliente' do arquivo CSV e converte para uma lista
clientes = pd.read_csv('clientes_elegiveis.csv')['nu_cliente'].tolist()

In [4]:
# Remove duplicatas da lista de cpfs e clientes
cpfs = np.unique(cpfs)
clientes = np.unique(clientes)

In [5]:
# Verifica se o CPF e cliente estão cadastrados e se o valor é positivo
from typing import List

def valida_transacao(cpf: int, cpf_list: List[int], cliente: int,
                     cliente_list: List[int], valor: int) -> bool:
    '''
    Checa requisitos para retonar se uma transação é válida ou não.

    Parâmetros:
        cpf: CPF da transação
        cpf_list: Lista de CPFs cadastrados
        cliente: Cliente da transação
        cliente_list: Lista de clientes cadastrados
        valor: Valor da transação
    Retorno booleano
    '''
    if cpf in cpf_list and cliente in cliente_list and valor > 0:
        return True
    else:
        return False

In [6]:
# Inicializa listas para armazenar transações válidas e erros
transacoes = []
erros = []

count = 0

# Abre o arquivo CSV para leitura
with open('payload_transacoes.csv', 'r') as f:
    for line in f:
        # Remove espaços em branco e divide a linha em partes
        s = line.strip()
        s = s.split(';')
        count += 1
        try:
            # Tenta converter o CPF para inteiro
            s[0] = int(s[0])
        except ValueError:
            # Adiciona um erro se CPF não for numérico
            erros.append([count, 'CPF precisa ser numerico.'])
            continue
        try:
            # Tenta converter o cliente para inteiro
            s[1] = int(s[1])
        except ValueError:
            # Adiciona um erro se o cliente não for numérico
            erros.append([count, 'NU_CLIENTE precisa ser numerico.'])
            continue
        try:
            # Tenta converter o valor da transação para inteiro
            s[2] = int(s[2])
        except ValueError:
            # Adiciona um erro se o valor não for numérico
            erros.append([count, 'Valor da transacao precisa ser numerico.'])
            continue
        try:
            # Verifica se o produto é uma string
            s[3] = str(s[3])
        except ValueError:
            # Adiciona um erro se o produto não for uma string
            erros.append([count, 'Produto precisa ser string.'])
            continue
        # Verifica se o produto é 'Streaming' ou 'Delivery'
        if s[3] not in (['Streaming', 'Delivery']):
            erros.append([count, 'Produto precisa ser streaming ou delivery.'])
            continue
        # Valida a transação
        if valida_transacao(s[0], cpfs, s[1], clientes, s[2]):
            # Adiciona a transação válida à lista
            transacoes.append([count, *s])
        else:
            # Adiciona um erro se a transação for inválida
            erros.append([count, 'CPF, NU_CLIENTE ou valor da transacao invalido(s).'])

# Imprime as primeiras cinco transações válidas e erros
print(transacoes[1:5])
print(erros[1:5])

[[6, 77, 4900, 24, 'Streaming'], [26, 28, 1659, 109, 'Streaming'], [27, 76, 3553, 130, 'Streaming'], [30, 98, 1279, 186, 'Streaming']]
[[3, 'CPF, NU_CLIENTE ou valor da transacao invalido(s).'], [4, 'CPF, NU_CLIENTE ou valor da transacao invalido(s).'], [5, 'CPF precisa ser numerico.'], [7, 'CPF, NU_CLIENTE ou valor da transacao invalido(s).']]


In [7]:
# Extrai os números das linhas válidas das transações
linhas_validas = [t[0] for t in transacoes]

# Extrai os números das linhas inválidas dos erros
linhas_invalidas = [t[0] for t in erros]

In [9]:
# Abre o arquivo para escrita
with open('transacoes_report_solucao.txt', 'w') as f:
    # Escreve o número de transações válidas
    f.write(f'{len(linhas_validas)} transacoes validas\n')
    # Escreve o número de transações inválidas
    f.write(f'{len(linhas_invalidas)} transacoes invalidas\n')
    # Itera sobre todas as linhas processadas
    for i in range(count+1):
        if i in linhas_validas:
            # Escreve mensagem para transações válidas
            f.write(f'Transacao {i} VALIDA.\n')
        elif i in linhas_invalidas:
            # Escreve mensagem para transações inválidas com a mensagem de erro correspondente
            f.write(f'Transacao {i} INVALIDA. {[e[1] for e in erros if e[0] == i][0]}\n')